# ----Lightgbm-----


In [10]:
import os
os.environ['LIGHTGBM_SUPPRESS_WARNINGS'] = '1'
os.environ['PYTHONWARNINGS'] = 'ignore'

In [ ]:
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr
import numpy as np
import joblib
import os
from lightgbm import early_stopping, log_evaluation
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
from sklearn.inspection import permutation_importance
import time


# Configuration
input_filename = 'Spotify_Model_Ready_Features_V2.csv'
use_gpu = True 

# split dates
TRAIN_END = pd.Timestamp('2021-12-31')
VAL_END = pd.Timestamp('2022-12-31')

BASELINE_FEATURES = [
    'Danceability', 'Energy', 'Loudness_Corrected', 'Speechiness',
    'Acousticness', 'Instrumentalness', 'Valence', 'Artist_Count',
    'Nationality_Count', 'Rank', 'Points (Total)', 'Rank_last_week',
    'Points_last_week', 'Rank_change', 'Points_change',
    'Points_rolling_mean_4w', 'Rank_rolling_mean_4w',
    'Weeks_on_chart', 'Artist_Hotness'
]
CHRISTMAS_FEATURES = BASELINE_FEATURES + ['is_christmas']


def mark_christmas_period(date_series: pd.Series) -> pd.Series:
    """Mark December and first week of January as Christmas period."""
    december = date_series.dt.month == 12
    january_first_week = (date_series.dt.month == 1) & (date_series.dt.day <= 7)
    return (december | january_first_week).astype(int)


def train_regression_pipeline(
    df_train, df_val, df_oot, feature_columns, target_column, model_params,
    model_name="", save_detailed_predictions=False
):
    """
    Train a LightGBM regression model with Christmas period segmentation.
    
    Data splits:
    - df_train: Training data (used to fit model)
    - df_val: Validation data (used for early stopping)
    - df_test: Test data (final evaluation,)
    
    Evaluates performance on both val and test sets, with segment breakdown:
    - All data
    - Christmas weeks only
    - Non-Christmas weeks only
    """
    try:
        output_dir = os.path.join("results", "regression", "Christmas_Model", target_column, model_name)
        os.makedirs(output_dir, exist_ok=True)
        
        suffix = target_column.replace('Points_', '')
        
        metrics_output_filename = os.path.join(output_dir, f"metrics_{suffix}_oot.csv")
        importance_output_filename = os.path.join(output_dir, f"importance_{suffix}.csv")
        model_output_filename = os.path.join(output_dir, f"model_{suffix}.pkl")
        oot_predictions_output_filename = os.path.join(output_dir, f"oot_predictions_and_actuals.csv")
        rank_metrics_filename = os.path.join(output_dir, f"metrics_derived_rank_oot.csv")
        

        print(f"\nStep 5: Training model for {target_column} ({model_name})")
    
        # Prepare training data
        df_train_target = df_train.dropna(subset=[target_column]).copy()
        X_train = df_train_target[feature_columns]
        y_train = df_train_target[target_column]
        
        if X_train.empty:
            print(f"Skipping {target_column}: No training data available after dropna.")
            return
        
        # Prepare validation data (for early stopping)
        df_val_target = df_val.dropna(subset=[target_column]).copy()
        X_val = df_val_target[feature_columns]
        y_val = df_val_target[target_column]
        
        if X_val.empty:
            print(f"Warning: Validation set for {target_column} is empty.")
            return
        
        # Train model with validation set for early stopping
        start_time = time.time()
        final_model = lgb.LGBMRegressor(**model_params)
        
        # Reuse static callbacks to reduce overhead
        if not hasattr(train_regression_pipeline, "_callbacks"):
            train_regression_pipeline._callbacks = [
                early_stopping(stopping_rounds=30),
                log_evaluation(period=0)
            ]
        callbacks = train_regression_pipeline._callbacks
        
        final_model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric="mae",
            callbacks=callbacks,
        )
        end_time = time.time()
        train_duration = end_time - start_time
        runtime_log_path = os.path.join(output_dir, f"runtime_log_{suffix}.csv")
        pd.DataFrame({
            'model': [model_name],
            'target': [target_column],
            'train_time_sec': [train_duration]
        }).to_csv(runtime_log_path, sep=';', index=False)
        print(f" Training duration logged to '{runtime_log_path}' ({train_duration:.1f} sec)")
        
        train_curve = final_model.evals_result_
        if train_curve and 'training' in train_curve and 'l1' in train_curve['training']:
            curve_df = pd.DataFrame({
                'iteration': range(len(train_curve['training']['l1'])),
                'train_mae': train_curve['training']['l1'],
                'val_mae': train_curve['valid_0']['l1']
            })
            curve_path = os.path.join(output_dir, f"training_curve_{suffix}.csv")
            curve_df.to_csv(curve_path, sep=';', index=False)
            print(f"Training curve saved to '{curve_path}'")
        
        try:
            device_type = final_model.booster_.params.get("device_type", "unknown")
            print(f"Training complete. Best iteration: {final_model.best_iteration_} | Device used: {device_type}")
        except Exception as e:
            print("Unable to detect device type:", e)

        joblib.dump(final_model, model_output_filename)
        print(f"Final model saved to '{model_output_filename}'")
        
        print("\nStep 7: Performing Out-of-Time (OOT) Hold-Out Testing")

        df_oot_target = df_oot.dropna(subset=[target_column]).copy()
        X_oot = df_oot_target[feature_columns]
        y_oot = df_oot_target[target_column]

        if X_oot.empty:
            print(f"Warning: OOT set for {target_column} is empty. Skipping OOT evaluation.")
            return

        oot_predictions = final_model.predict(X_oot)

        # Derive and Evaluate Ranks from Points Predictions
        print("\n--- Deriving and Evaluating Ranks from Points Predictions ---")

        # 创建 DataFrame 保留日期，用于每周分组排序
        results_df = df_oot_target[['Date']].copy()
        results_df['true_points'] = y_oot
        results_df['predicted_points'] = oot_predictions

        # 获取真实排名列用于对比
        true_rank_col_name = target_column.replace('Points', 'Rank')
        results_df['true_rank'] = df_oot_target[true_rank_col_name]

        # 按每周积分降序排序生成预测排名
        results_df['predicted_rank'] = results_df.groupby('Date')['predicted_points'].rank(
            method='first', ascending=False
        )

        # 计算与积分一致的四个指标
        mae_rank = mean_absolute_error(results_df['true_rank'], results_df['predicted_rank'])
        rmse_rank = np.sqrt(mean_squared_error(results_df['true_rank'], results_df['predicted_rank']))
        r2_rank = r2_score(results_df['true_rank'], results_df['predicted_rank'])
        spearman_rank, _ = spearmanr(results_df['true_rank'], results_df['predicted_rank'])

        print("\n--- Derived Rank Evaluation (Aligned with Points Metrics) ---")
        print(f"MAE (Rank): {mae_rank:.3f}")
        print(f"RMSE (Rank): {rmse_rank:.3f}")
        print(f"R² (Rank): {r2_rank:.3f}")
        print(f"Spearman (Rank): {spearman_rank:.3f}")

        # Compute metrics
        mae_oot = mean_absolute_error(y_oot, oot_predictions)
        r2_oot = r2_score(y_oot, oot_predictions)
        spearman_oot, _ = spearmanr(y_oot, oot_predictions)
        rmse_oot = np.sqrt(mean_squared_error(y_oot, oot_predictions))
        
        def compute_metric_std(y_true, y_pred, n_splits=3):
            size = len(y_true) // n_splits
            maes, r2s, rmses, spearmans = [], [], [], []
            for i in range(n_splits):
                start, end = i * size, (i + 1) * size
                y_t, y_p = y_true[start:end], y_pred[start:end]
                if len(y_t) == 0: continue
                maes.append(mean_absolute_error(y_t, y_p))
                r2s.append(r2_score(y_t, y_p))
                rmses.append(np.sqrt(mean_squared_error(y_t, y_p)))
                # Handle potential errors in Spearman calculation on small chunks
                if len(np.unique(y_t)) > 1 and len(np.unique(y_p)) > 1:
                    spearmans.append(spearmanr(y_t, y_p)[0])
                else:
                    spearmans.append(np.nan)
            
            return {
                'MAE_std': np.nanstd(maes),
                'R2_std': np.nanstd(r2s),
                'RMSE_std': np.nanstd(rmses),
                'Spearman_std': np.nanstd(spearmans)
            }
        
        metric_std = compute_metric_std(y_oot.values, oot_predictions, n_splits=3)

        print("\n--- OOT Hold-Out Results ---")
        print(f"MAE: {mae_oot:.2f} ± {metric_std['MAE_std']:.2f}")
        print(f"RMSE: {rmse_oot:.2f} ± {metric_std['RMSE_std']:.2f}")
        print(f"R²: {r2_oot:.2f} ± {metric_std['R2_std']:.2f}")
        print(f"Spearman Corr: {spearman_oot:.2f} ± {metric_std['Spearman_std']:.2f}")

        oot_results_df = pd.DataFrame({
            'Metric': ['MAE', 'RMSE', 'R2', 'Spearman'],
            'Mean': [mae_oot, rmse_oot, r2_oot, spearman_oot],
            'Std': [
                metric_std['MAE_std'],
                metric_std['RMSE_std'],
                metric_std['R2_std'],
                metric_std['Spearman_std']
            ]
        })
        oot_results_df.to_csv(metrics_output_filename, index=False, sep=';')

        # 使用 tabulate 打印更漂亮的表格，不改变任何变量
        from tabulate import tabulate
        print("\n=== OOT Hold-Out Summary (Mean ± Std) ===")
        print(tabulate(
            oot_results_df,
            headers="keys",
            tablefmt="psql", 
            floatfmt=".3f"
        ))

        print(f"\nEvaluation summary saved to:")
        print(f"  - Metrics CSV:      {metrics_output_filename}")
        print(f"  - Rank metrics CSV: {rank_metrics_filename}")
        print(f"  - Importance CSV:   {importance_output_filename}")
        if save_detailed_predictions:
            detailed_output_filename = os.path.join(output_dir, f"oot_predictions_detailed_{suffix}.csv")
            print(f"  - Detailed preds:   {detailed_output_filename}")


        # Save simple predictions for visualization
        oot_output_df = pd.DataFrame({'y_true': y_oot, 'y_pred': oot_predictions})
        oot_output_df.to_csv(oot_predictions_output_filename, index=False, sep=';')

        residuals = y_oot - oot_predictions
        residuals_df = pd.DataFrame({
            'Date': df_oot_target['Date'],
            'y_true': y_oot,
            'y_pred': oot_predictions,
            'residual': residuals
        })
        residuals_path = os.path.join(output_dir, f"residuals_{suffix}.csv")
        residuals_df.to_csv(residuals_path, sep=';', index=False)
        print(f"Residuals saved to '{residuals_path}'")


        # 预测置信区间 (95%)
        residual_std = np.std(residuals)
        ci_lower = oot_predictions - 1.96 * residual_std
        ci_upper = oot_predictions + 1.96 * residual_std

        ci_df = pd.DataFrame({
            'y_pred': oot_predictions,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper
        })
        ci_path = os.path.join(output_dir, f"confidence_interval_{suffix}.csv")
        ci_df.to_csv(ci_path, sep=';', index=False)
        print(f" Prediction confidence interval saved to '{ci_path}' (σ={residual_std:.3f})")


        # Save feature importance
        feature_importance_df = pd.DataFrame({
            'feature': feature_columns,
            'importance': final_model.feature_importances_
        }).sort_values('importance', ascending=False).reset_index(drop=True)
        feature_importance_df.to_csv(importance_output_filename, index=False, sep=';')

        print(f"Feature importance saved to '{importance_output_filename}'")

                #  Compute model-agnostic Permutation Feature Importance 
        try:
            print("\n=== Computing Permutation Feature Importance (Model-Agnostic) ===")
            perm_output_filename = os.path.join(output_dir, f"importance_permutation_{suffix}.csv")

            # 仅在 OOT 集上计算，以避免过拟合影响
            perm_result = permutation_importance(
                final_model, X_oot, y_oot,
                scoring='neg_mean_absolute_error',
                n_repeats=10,
                random_state=42
            )

            perm_importance_df = pd.DataFrame({
                'feature': feature_columns,
                'importance_mean': perm_result.importances_mean,
                'importance_std': perm_result.importances_std
            }).sort_values('importance_mean', ascending=False)

            perm_importance_df.to_csv(perm_output_filename, sep=';', index=False)
            print(f"Permutation importance saved to '{perm_output_filename}'")

        except Exception as e:
            print(f"Permutation importance computation failed: {e}")

        
        # It saves the detailed, per-sample predictions needed for a paired t-test
        if save_detailed_predictions:
            # include the Date and true target for pairing
            detailed_preds_df = df_oot_target[['Date', target_column]].copy()
            detailed_preds_df['y_pred'] = oot_predictions
            detailed_preds_df['model_name'] = model_name
            
            detailed_output_filename = os.path.join(output_dir, f"oot_predictions_detailed_{suffix}.csv")
            detailed_preds_df.to_csv(detailed_output_filename, index=False, sep=';')
            print(f"Detailed predictions for significance testing saved to '{detailed_output_filename}'")


        print(f" All results and data for {target_column} ({model_name}) saved to '{output_dir}'")

        return pd.DataFrame({'y_true': y_oot, 'y_pred': oot_predictions})

    except Exception as e:
        print(f"An error occurred during processing for {target_column} ({model_name}): {e}")


def hyperparameter_search(X, y, is_classification=False):
    print(f"\nStarting Hyperparameter Search (Mode: {'Classification' if is_classification else 'Regression'})")
    
    param_dist = {
        "max_depth": [6, 8, 10, 12],
        "num_leaves": [32, 64, 128, 256],
        "learning_rate": [0.03, 0.05, 0.08, 0.1],
        "n_estimators": [300],
        "reg_lambda": [3, 5, 7, 9],
        "subsample": [0.7, 0.8, 0.9, 1.0],
        "colsample_bytree": [0.7, 0.8, 0.9, 1.0],
        "random_state": [42],
    }
    
    N_ITER = 20  
    N_SPLITS = 3
    total_fits = N_ITER * N_SPLITS
    device = 'gpu' if use_gpu else 'cpu'
    
    base_estimator = lgb.LGBMRegressor(
        objective='regression',
        metric='mae',
        random_state=42,
        n_jobs=1,  
        device=device,
        verbose=-1,
    )
    

    # Split for early stopping validation
    split_idx = int(len(X) * 0.9)
    X_search, X_val_search = X.iloc[:split_idx], X.iloc[split_idx:]
    y_search, y_val_search = y.iloc[:split_idx], y.iloc[split_idx:]

    random_search = RandomizedSearchCV(
        estimator=base_estimator,
        param_distributions=param_dist,
        n_iter=N_ITER,
        scoring='neg_mean_absolute_error',
        cv=TimeSeriesSplit(n_splits=N_SPLITS),
        n_jobs=-1,  
        random_state=42,
        verbose=2,  
    )

    print("\nFitting RandomizedSearchCV with early stopping...")
    callbacks = [
        early_stopping(stopping_rounds=30),
        log_evaluation(period=0)
    ]

    random_search.fit(
        X_search, y_search,
        eval_metric='mae',
        eval_set=[(X_val_search, y_val_search)],  
        callbacks=callbacks
    )


    print("\nBest parameters found:")
    print(random_search.best_params_)
    return random_search.best_params_


# ==============================
# Main Execution
# ==============================
if __name__ == "__main__":
    # GPU configuration
    os.environ["LIGHTGBM_USE_GPU"] = "1"
    os.environ["GPU_PLATFORM_ID"] = "0"
    os.environ["GPU_DEVICE_ID"] = "0"
    os.environ["LIGHTGBM_DEBUG_VERBOSE"] = "0"
    
    # GPU Pre-flight Check
    print("Step 0: Performing GPU Pre-flight Check...")
    try:
        dummy_X = np.random.rand(10, 5)
        dummy_y = np.random.rand(10)
        dummy_model = lgb.LGBMRegressor(device='gpu')

        import warnings
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            dummy_model.fit(dummy_X, dummy_y)
        print(" GPU Pre-flight Check PASSED. LightGBM can access the GPU.")
        
    except lgb.basic.LightGBMError as e:
        print(" GPU Pre-flight Check FAILED.")
        print("   Error:", e)
        print("   Please ensure LightGBM was installed with GPU support and that drivers are correct.")
        print("   The script will now exit.")
        exit()
    except Exception as e:
        print(f"An unexpected error occurred during GPU check: {e}")
        exit()


    print("\n\n== RUNNING REGRESSION MODELS IN OOT VALIDATION MODE ===")
    try:
        df = pd.read_csv(input_filename, sep=';', parse_dates=['Date'])
        df.sort_values('Date', inplace=True)
    except FileNotFoundError:
        print(f"CRITICAL ERROR: Input file '{input_filename}' not found. Exiting.")
        exit()


    # Add is_christmas feature if not present
    if 'is_christmas' not in df.columns:
        df['is_christmas'] = mark_christmas_period(df['Date'])
        print("Added 'is_christmas' feature to dataset")
    else:
        df['is_christmas'] = df['is_christmas'].astype(int)
        print("'is_christmas' feature already present in dataset")
    
    # Fill missing values in lag features
    lag_fill_columns = [
        'Rank_last_week', 'Points_last_week', 'Rank_change', 'Points_change',
        'Points_rolling_mean_4w', 'Rank_rolling_mean_4w', 'Artist_Hotness'
    ]
    existing_fill = [col for col in lag_fill_columns if col in df.columns]
    df[existing_fill] = df[existing_fill].fillna(0)
    
    # split train / val / test
    train_df = df[df['Date'] <= TRAIN_END].copy()
    val_df = df[(df['Date'] > TRAIN_END) & (df['Date'] <= VAL_END)].copy()
    test_df = df[df['Date'] > VAL_END].copy()
    

    print(f"  Train: <= {TRAIN_END.date()} ({train_df.shape[0]} rows)")
    print(f"  Val:   {TRAIN_END.date()} < Date <= {VAL_END.date()} ({val_df.shape[0]} rows)")
    print(f"  Test:  > {VAL_END.date()} ({test_df.shape[0]} rows)")
    print(f"\nChristmas weeks distribution:")
    print(f"  Train: {train_df['is_christmas'].sum()} / {len(train_df)} ({100*train_df['is_christmas'].sum()/len(train_df):.1f}%)")
    print(f"  Val:   {val_df['is_christmas'].sum()} / {len(val_df)} ({100*val_df['is_christmas'].sum()/len(val_df):.1f}%)")
    print(f"  Test:  {test_df['is_christmas'].sum()} / {len(test_df)} ({100*test_df['is_christmas'].sum()/len(test_df):.1f}%)")
    
    # Define feature sets for comparison
    feature_sets = {
        'baseline': BASELINE_FEATURES,
        'baseline_plus_christmas': CHRISTMAS_FEATURES,
    }
    
    # Define regression targets
    regression_targets = ['Points_next_week']
    
    # Cache cleaned datasets to avoid repeated dropna
    cached_targets = {}
    for target in regression_targets:
        for feature_set_name, feature_cols in feature_sets.items():
            subset = train_df.dropna(subset=[target])
            if not subset.empty:
                cache_key = f"{target}_{feature_set_name}"
                cached_targets[cache_key] = (subset[feature_cols], subset[target])
    print(f"Cached {len(cached_targets)} training subsets")
    
    # Loop
    for target in regression_targets:
        print(f"\n\n===== Processing Regression Target: {target} =====")
        
        for feature_set_name, feature_cols in feature_sets.items():
            cache_key = f"{target}_{feature_set_name}"
            
            if cache_key not in cached_targets:
                print(f"\nSkipping {target} with {feature_set_name}: No training data after dropna.")
                continue
            
            print(f"\n{'─'*80}")
            print(f"Feature Set: {feature_set_name}")
            print(f"Features: {len(feature_cols)} ({feature_cols[-1] if feature_set_name == 'baseline_plus_christmas' else 'baseline only'})")
            print(f"{'─'*80}")
            
            X_all, y_all = cached_targets[cache_key]
            
            # Hyperparameter search
            print(f"\n--- Hyperparameter Search for {feature_set_name} ---")
            best_params = hyperparameter_search(X_all, y_all, is_classification=False)
            
            # Add fixed seeds and GPU config
            best_params['random_state'] = 42
            best_params['device'] = 'gpu' if use_gpu else 'cpu'
            best_params['bagging_seed'] = 42
            best_params['feature_fraction_seed'] = 42
            best_params['n_estimators'] = 1800
            
            # Train and evaluate model
            train_regression_pipeline(
                train_df,
                val_df,
                test_df,
                feature_cols,
                target,
                best_params,
                model_name=feature_set_name,
                save_detailed_predictions=True,
            )
    
    print("CHRISTMAS MODEL PIPELINE COMPLETE")
    print("\nComparison between baseline and baseline_plus_christmas available for:")
    for target in regression_targets:
        print(f"  - {target}")
    print("\nDetailed predictions saved for significance testing.")

Step 0: Performing GPU Pre-flight Check...
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 10, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 4060 Laptop GPU, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.471304
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM